<a href="https://colab.research.google.com/github/hardayal/EVA/blob/master/Session_20/EVA_Assignment_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Used ResNet18 Pre-trained model and freezed the FC layers using include_top = False
### Resized the CIFAR100 dataset to size 224x224.


In [1]:
from keras.layers import GlobalAveragePooling2D,Activation,Flatten,Conv2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
import keras.backend as K
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import np_utils
import matplotlib.pyplot as plt
import skimage.transform

Using TensorFlow backend.


In [2]:
pip install image-classifiers

In [3]:
pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-gqcf3zx3
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-gqcf3zx3
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=19950 sha256=f6713e283c05f60456b2e40b7d0a52e27726b600f18c113838bcaae83ad46da0
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ie5fle4/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [0]:
from classification_models.keras import Classifiers 
ResNet18, preprocess_input = Classifiers.get('resnet18') #from classification_models.resnet import ResNet18, preprocess_input
model = ResNet18((224, 224, 3), weights='imagenet',include_top=False)

In [6]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_19[0][0]          
____________________________________________________________________________________________

In [0]:
num_classes = 100
batch_size = 128
epochs = 5

In [0]:
# add a global spatial average pooling layer
last_layer = model.output
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(last_layer)
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(x)
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(x)
x = Conv2D(num_classes, (1,1), strides=(1,1), use_bias=False)(last_layer)
x = GlobalAveragePooling2D()(x)
# a softmax layer for 10 classes
out = Activation('softmax')(x)

In [0]:
# Model we will train
custom_resnet_model = Model(inputs=model.input, outputs=out)

In [10]:
custom_resnet_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_19[0][0]          
____________________________________________________________________________________________

In [11]:
custom_resnet_model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
#x_train = x_train / 255.0
#x_test = x_test / 255.0
y_train = np_utils.to_categorical(y_train, 100)
y_test = np_utils.to_categorical(y_test, 100)

169009152/169001437 [==============================] - 4s 0us/step


In [0]:
new_shape = (224,224)
x_train = np.asarray([cv2.resize(image, new_shape) for image in x_train])
x_test = np.asarray([cv2.resize(image, new_shape) for image in x_test])

In [14]:
x_test.shape

(10000, 224, 224, 3)

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Defining the path to store the weights
filepath="/content/gdrive/My Drive/Hardayal_EVA-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

In [17]:
custom_resnet_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15,
              validation_data=(x_test, y_test),
              shuffle=False,
              callbacks=[checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 138s 3ms/step - loss: 3.3844 - acc: 0.2433 - val_loss: 2.2668 - val_acc: 0.4337

Epoch 00001: val_acc improved from -inf to 0.43370, saving model to /content/gdrive/My Drive/Hardayal_EVA-01-0.43.hdf5
Epoch 2/15
50000/50000 [==============================] - 129s 3ms/step - loss: 1.8007 - acc: 0.5474 - val_loss: 1.5592 - val_acc: 0.5823

Epoch 00002: val_acc improved from 0.43370 to 0.58230, saving model to /content/gdrive/My Drive/Hardayal_EVA-02-0.58.hdf5
Epoch 3/15
50000/50000 [==============================] - 129s 3ms/step - loss: 1.2919 - acc: 0.6603 - val_loss: 1.2782 - val_acc: 0.6482

Epoch 00003: val_acc improved from 0.58230 to 0.64820, saving model to /content/gdrive/My Drive/Hardayal_EVA-03-0.65.hdf5
Epoch 4/15
50000/50000 [==============================] - 129s 3ms/step - 